<a href="https://colab.research.google.com/github/melzismn/Digital-Design-2020-2021/blob/master/Simplification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ONLY FOR COLAB
# Not required in Binder

!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

!conda install -q -y --prefix /usr/local python=3.6 ujson

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

import ujson
print(ujson.dumps({1:2}))

!conda install -c conda-forge igl
!conda install -c conda-forge meshplot

In [ ]:
 !pip install --upgrade --force-reinstall Pillow

import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans

# Read and visualize meshes

There are several libraries to do Geometry Processing in Python. In this case we will use the python binding of [libigl](https://libigl.github.io/libigl-python-bindings/). Another nice library for 3D visualization is [Open3D](http://www.open3d.org/).

In [ ]:
# Read and plot mesh
v, f = igl.read_triangle_mesh(os.path.join('.', "data", "bunny.off"))
plot(v, f)

# **Compute the edge list**
Once you have the edge list you can compute the number of vertices, the number of edges and the number of faces.
Then you can check all the fiormula that we see during the lecture.


In [ ]:
e_raw = np.concatenate((np.concatenate((np.expand_dims(f[:,0],axis=1),np.expand_dims(f[:,1],axis=1)),axis=1),np.concatenate((np.expand_dims(f[:,1],axis=1),np.expand_dims(f[:,2],axis=1)),axis=1),np.concatenate((np.expand_dims(f[:,2],axis=1),np.expand_dims(f[:,0],axis=1)),axis=1)), axis=0)
e = np.unique(np.sort(e_raw,axis=1), axis=0)
nv = v.shape[0]
ne = e.shape[0]
nf = f.shape[0]


# **Implement the edge contraction**
The simplest implementation

In [ ]:
errors = np.zeros((nv,1))
vnew = v;
fnew = f;
enew = e;
nvnew = nv
nenew = ne
nv_to_remove = 500
 
    
for i in range(0,nv_to_remove):
    edge_length = np.linalg.norm(vnew[enew[:,0],:]-vnew[enew[:,1],:],axis=1)
    
    shorter = np.argmin(edge_length)
    v_to_remove = enew[shorter,0]
    v_to_save = enew[shorter,1]
    
    vnew = np.delete(vnew, v_to_remove, 0)
    fnew[fnew == v_to_remove] = v_to_save
    
    mask = np.sum(fnew == v_to_save, axis=1)
    fnew = np.delete(fnew, mask>1, 0)
    
    fnew[fnew > v_to_remove] -=1
    enew_raw = np.concatenate((np.concatenate((np.expand_dims(fnew[:,0],axis=1),np.expand_dims(fnew[:,1],axis=1)),axis=1),np.concatenate((np.expand_dims(fnew[:,1],axis=1),np.expand_dims(fnew[:,2],axis=1)),axis=1),np.concatenate((np.expand_dims(fnew[:,2],axis=1),np.expand_dims(fnew[:,0],axis=1)),axis=1)), axis=0)
    enew = np.unique(np.sort(enew_raw,axis=1), axis=0)

plot(vnew, fnew, shading={"wireframe": True})



In [ ]:
[v.shape[0], vnew.shape[0]]